# Related Works

In [1]:
dir1 = 'Videos\\Video1.avi'
dir2 = 'Videos\\Video2.avi'

In [2]:
# importing the necessary libraries 
import cv2 
import numpy as np

## Display Video

In [3]:
# Creating a VideoCapture object to read the video 
cap = cv2.VideoCapture(dir1)   
  
# Loop untill the end of the video 
while (cap.isOpened()): 
  
    # Capture frame-by-frame 
    ret, frame = cap.read() 
    frame = cv2.resize(frame, (540, 380), fx = 0, fy = 0, 
                         interpolation = cv2.INTER_CUBIC) 
  
    # Display the resulting frame 
    cv2.imshow('Frame', frame) 
  
    # conversion of BGR to grayscale is necessary to apply this operation 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
  
    # adaptive thresholding to use different threshold  
    # values on different regions of the frame. 
    Thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                           cv2.THRESH_BINARY_INV, 11, 2) 
  
    cv2.imshow('Thresh', Thresh) 
    # define q as the exit button 
    if cv2.waitKey(25) & 0xFF == ord('q'): 
        break
  
# release the video capture object 
cap.release() 
# Closes all the windows currently opened. 
cv2.destroyAllWindows() 

## Backgraound Subtractor

In [3]:
from cv2 import bgsegm
cap = cv2.VideoCapture(dir1) 

Background_subtract = cv2.bgsegm.createBackgroundSubtractorMOG()

while (1):
    #ret and frame, first and next frame
    ret,frame = cap.read()
    first_frame = Background_subtract.apply(frame)
    gblur = cv2.GaussianBlur(first_frame, (5, 5), 0)

    #contouring
    #threshold : Separate out regions of an image corresponding to objects which we want to analyze.
    ret, threshold = cv2.threshold(gblur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    _, contours, _ = cv2.findContours(threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
#     print("number of contours detected", len(contours))

    cv2.drawContours(frame, contours, 0, (0, 0, 255), 6)

    cv2.imshow('original', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'): 
        break
    
    cv2.imshow('first frame', gblur)
    if cv2.waitKey(25) & 0xFF == ord('q'): 
        break

cap.release()
cv2.waitKey()
cv2.destroyAllWindows()

## Background Subtraction Using Median Filter

In [2]:
import numpy as np
import cv2
from skimage import data, filters

# Open Video
cap = cv2.VideoCapture(dir1)

# Randomly select 25 frames
frameIds = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=25)

# Store selected frames in an array
frames = []
for fid in frameIds:
    cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
    ret, frame = cap.read()
    frames.append(frame)

# Calculate the median along the time axis
medianFrame = np.median(frames, axis=0).astype(dtype=np.uint8)    

# Display median frame
cv2.imshow('frame', medianFrame)
cv2.waitKey(0)

# Reset frame number to 0
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Convert background to grayscale
grayMedianFrame = cv2.cvtColor(medianFrame, cv2.COLOR_BGR2GRAY)

# Loop over all frames
ret = True
while(ret):

    # Read frame
    ret, frame = cap.read()
    # Convert current frame to grayscale
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Calculate absolute difference of current frame and 
    # the median frame
    dframe = cv2.absdiff(frame, grayMedianFrame)
    # Treshold to binarize
    th, dframe = cv2.threshold(dframe, 30, 255, cv2.THRESH_BINARY)
    # Display image
    cv2.imshow('frame', dframe)
    cv2.waitKey(20)
    key = cv2.waitKey(30)
    if key == 27:
        break

# Release video object
cap.release()

# Destroy all windows
cv2.destroyAllWindows()

## Vehicle detection Using HAAR Features

In [8]:
import matplotlib.pyplot as plt

video_src = dir1

xml_car = 'XMLs\\cars.xml'

cap = cv2.VideoCapture(video_src)

car_cascade = cv2.CascadeClassifier(xml_car)

while True:
    ret, img = cap.read()
    
    if (type(img) == type(None)):
        break
#     img = remove_bg(img)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
#     plt.imshow(gray),plt.show()
    
    cars = car_cascade.detectMultiScale(gray, 1.3, 2)
    
    for (x,y,w,h) in cars:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)    
        
    cv2.imshow('video', img)
    
    if cv2.waitKey(33) == 27:
        break

cv2.destroyAllWindows()

## Blob Detection

In [ ]:
from cv2 import bgsegm
import cv2

cap = cv2.VideoCapture(dir1) 

# Set up the detector with default parameters.
detector = cv2.SimpleBlobDetector()

Background_subtract = cv2.bgsegm.createBackgroundSubtractorMOG()

while (1):
    #ret and frame, first and next frame
    ret,frame = cap.read()
    first_frame = Background_subtract.apply(frame)
    gblur = cv2.GaussianBlur(first_frame, (5, 5), 0)
    gblur = cv2.cvtColor(gblur, cv2.COLOR_BGR2GRAY)

    #contouring
    #threshold : Separate out regions of an image corresponding to objects which we want to analyze.
    ret, threshold = cv2.threshold(gblur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    _, contours, _ = cv2.findContours(threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    print("number of contours detected", len(contours))

    for cnt in contours:
        area = cv2.contourArea(cnt)

    cv2.drawContours(frame, contours, 0, (0, 0, 255), 6)

    cv2.imshow('original', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'): 
        break
        
    # Detect blobs.
    keypoints = detector.detect(gblur)
    im_with_keypoints = cv2.drawKeypoints(gblur, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imshow('first frame', gblur)
    if cv2.waitKey(25) & 0xFF == ord('q'): 
        break

cap.release()
cv2.waitKey()
cv2.destroyAllWindows()

## Remove Background

In [6]:
import cv2
import numpy as np
def remove_bg(
    img,
    BLUR = 21,
    CANNY_THRESH_1 = 10,
    CANNY_THRESH_2 = 200,
    MASK_DILATE_ITER = 10,
    MASK_ERODE_ITER = 10,
    MASK_COLOR = (0.0,0.0,1.0),
):
#     img = cv2.imread(path)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    edges = cv2.Canny(gray, CANNY_THRESH_1, CANNY_THRESH_2)
    edges = cv2.dilate(edges, None)
    edges = cv2.erode(edges, None)

    contour_info = []
    _, contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    for c in contours:
        contour_info.append((
            c,
            cv2.isContourConvex(c),
            cv2.contourArea(c),
        ))
    contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
    max_contour = contour_info[0]

    mask = np.zeros(edges.shape)
    cv2.fillConvexPoly(mask, max_contour[0], (255))

    mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
    mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
    mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)
    mask_stack = np.dstack([mask]*3)    # Create 3-channel alpha mask

    mask_stack  = mask_stack.astype('float32') / 255.0          # Use float matrices, 
    img         = img.astype('float32') / 255.0                 #  for easy blending

    masked = (mask_stack * img) + ((1-mask_stack) * MASK_COLOR) # Blend
    masked = (masked * 255).astype('uint8')                     # Convert back to 8-bit 

    c_blue, c_green, c_red = cv2.split(img)

    img_a = cv2.merge((c_red, c_green, c_blue, mask.astype('float32') / 255.0))

    
#     plt.imshow(img_a)
#     plt.show()
    return img_a

## Tracking with Opencv Tools

In [3]:
# OpenCV Methods for Tracking
def ask_for_tracker():
    print('Hi!, What tracker API would you like to use?')
    print('Enter 0 for BOOSTING: ')
    print('Enter 1 for MIL: ')
    print('Enter 2 for KCF: ')
    print('Enter 3 for TLD: ')
    print('Enter 4 for MEDIANFLOW: ')
    print('Enter 5 for GOTURN: ')
    print('Enter 6 for MOSSE: ')
    print('Enter 7 for CSRT: ')
    
    choice = input('Please select your tracker: ')
    
    if choice == '0':
        tracker = cv2.TrackerBoosting_create()
    if choice == '1':
        tracker = cv2.TrackerMIL_create()
    if choice == '2':
        tracker = cv2.TrackerKCF_create()
    if choice == '3':
        tracker = cv2.TrackerTLD_create()
    if choice == '4':
        tracker = cv2.TrackerMedianFlow_create()
    if choice == '5':
        tracker = cv2.TrackerGOTURN_create()
    if choice == '6':
        tracker = cv2.TrackerMOSSE_create()
    if choice == '7':
        tracker = cv2.TrackerCSRT_create()

    return tracker

In [ ]:
# Tracker
tracker = ask_for_tracker()

# Tracker name
tracker_name = str(tracker).split()[0][1:]

# Capture the Video
cap = cv2.VideoCapture(dir1)

# Read the first frame
ret, frame = cap.read()

# Select our ROI
roi = cv2.selectROI(frame, False)

# Initialize tracker
ret = tracker.init(frame, roi)

# while Loop
while True:

    # Read the capture
    ret, frame = cap.read()
    
    # update tracker
    success, roi = tracker.update(frame)
    
    # roi -> from tuple to int
    (x, y, w, h) = tuple(map(int, roi))
    
    # Draw rects as tracker moves
    if success:
        
        # Sucess on tracking
        pts1 = (x,y)
        pts2 = (x+w, y+h)
        cv2.rectangle(frame,
                     pts1,
                     pts2,
                     (255,125,25),
                     3)
    # else
    else:
    
        # Failure on tracking
        cv2.putText(frame, 
                   'Fail to tracthe object',
                   (100,200),
                   cv2.FONT_HERSHEY_SIMPLEX,
                   1,
                   (25, 125,255),
                   3)
        
    # Display Tracker
    cv2.putText(frame,
               tracker_name,
               (20,400),
               cv2.FONT_HERSHEY_SIMPLEX,
               1,
               (255,255,0),
               3)
    
    # Display result
    cv2.imshow(tracker_name, frame)
        
    # Exit with Esc button
    if cv2.waitKey(50) & 0xFF == 27:
        break
    
# Release the Capture & Destroy All Windows
cap.release()
cv2.destroyAllWindows()

## Balls Tracking

In [19]:
import cv2 as cv
import numpy as np
import random

class ball_game:
    
    def __init__(self):
        #Environment settings
        self.width = 1000
        self.height = 600
        self.limits = [0,0,self.width,self.height]
        #Past bounding boxes of the balls
        self.past_bboxes = []
        #Bounding boxes of the balls
        self.bboxes = []
        #Trackers of the balls
        self.trackers = []
        #Id count numbers
        self.ids_count = 0
        #Id numbers
        self.ids = []
        #Boxes to be tracked
        self.balls = []
        #Adding balls
        self.balls.append(self.create_ball([int(self.width/2),int(self.height/2)],[10,3],[True,True],50,(0,0,255)))
        self.balls.append(self.create_ball([int(self.width/3),int(self.height/3)],[10,6],[True,True],50,(0,255,0)))
        #Run Code
        self.run()
        
    def create_ball(self,positions,speeds,directions,size,color):
        ball = [positions,speeds,directions,size,color]
        self.create_tracker(ball)
        return ball
    
    def create_tracker(self,ball):
        tracker = [[self.ids_count],[ball[0][0],ball[0][1],ball[0][0]+2*ball[3],ball[0][1]+2*ball[3]]]
        self.trackers.append(tracker)
        self.ids.append(self.ids_count)
        self.ids_count = self.ids_count+1
    
    def balls_manager(self):
        for i,ball in enumerate(self.balls):
            x = ball[0][0]
            y = ball[0][1]
            size = ball[-2]        
            if x+size>=self.limits[2] or x-size<=self.limits[0]:
                ball[1][0] = ball[1][0]*-1
            if y+size>=self.limits[3] or y-size<=self.limits[1]:
                ball[1][1] = ball[1][1]*-1                   
        for i,ball in enumerate(self.balls):
            ball[0][0] = ball[0][0] + ball[1][0]
            ball[0][1] = ball[0][1] + ball[1][1]
                
    def draw_balls(self,frame):
        for ball in self.balls:
            cv.circle(frame,(ball[0][0],ball[0][1]),ball[-2],ball[-1],-1)
            cv.rectangle(frame,(int(ball[0][0]-ball[-2]/1),int(ball[0][1]-ball[-2]/1)),(int(ball[0][0]+ball[-2]/1),int(ball[0][1]+ball[-2]/1)),(255,0,0),3)
        for tracker in self.trackers:
            cv.putText(frame,str(tracker[0]),(int(tracker[1][0]+20),int(tracker[1][1]+50)),cv.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)
        for past_boxes in self.past_bboxes:
            cv.rectangle(frame,(int(past_boxes[0]),int(past_boxes[1])),(int(past_boxes[2]),int(past_boxes[3])),(100,100,100),2)
    
    def update_bboxes(self):
        self.past_bboxes = self.bboxes.copy()
        self.bboxes.clear()
        for ball in self.balls:
            self.bboxes.append([int(ball[0][0]-ball[-2]/1),int(ball[0][1]-ball[-2]/1),int(ball[0][0]+ball[-2]/1),int(ball[0][1]+ball[-2]/1)])
    
    def update_trackers(self):
        new_trackers = []
        overlaps_bboxes = []
        for k,present_box in enumerate(self.bboxes):
            overlaps_bbox = []
            for i,past_box in self.trackers:
                x_dif = 0
                y_dif = 0
                past_area = abs(past_box[0]-past_box[2])*abs(past_box[1]-past_box[3])
                if present_box[0]>=past_box[0] and present_box[0]<=past_box[2]:
                    x_dif = abs(present_box[0]-past_box[2])
                elif present_box[2]>=past_box[0] and present_box[2]<=past_box[2]:
                    x_dif = abs(present_box[2]-past_box[0])
                if present_box[1]>=past_box[1] and present_box[1]<=past_box[3]:
                    y_dif = abs(present_box[1]-past_box[3])
                elif present_box[3]>=past_box[1] and present_box[3]<=past_box[3]:
                    y_dif = abs(present_box[3]-past_box[1])
                overlap = x_dif*y_dif/past_area
                overlaps_bbox.append([overlap,i,k])
            overlaps_bboxes.append(overlaps_bbox)
        ids_selected = []
        for over_bbox in overlaps_bboxes:
            if max(over_bbox)[1][0] not in ids_selected:
                tracker = [max(over_bbox)[1],self.bboxes[max(over_bbox)[-1]]]
                new_trackers.append(tracker)
                ids_selected.append(max(over_bbox)[1][0])
                continue
            if max(over_bbox)[1][0] in ids_selected:
                id_num = list(set((self.ids))-set((ids_selected)))[0]
                tracker = [[id_num],self.bboxes[max(over_bbox)[-1]]]
                new_trackers.append(tracker)
                ids_selected.append(id_num)
                continue
        #print(overlaps_bboxes)
        self.trackers = new_trackers.copy()
    
    def run(self):
        while(True):
            background = np.zeros((self.height,self.width,3))
            self.balls_manager()
            self.draw_balls(background)
            self.update_bboxes()
            self.update_trackers()
            cv.imshow('frame',background)
            # Press Q on keyboard to  exit 
            if cv.waitKey(1) & 0xFF == ord('q'): 
                break
            
        cv.destroyAllWindows()
ball_game()

## Optical Flow

In [5]:
# Import Libraries
import cv2
import numpy as np

# Parameters for Shi-tomasi corner detection
st_params = dict(maxCorners=30,
                qualityLevel=0.2,
                minDistance=2,
                blockSize=7)

# Parameters for Lucas-Kande optical flow
lk_params = dict(winSize=(15,15),
                maxLevel=2,
                criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1))

# Video Capture
cap = cv2.VideoCapture(dir1)

# Color for optical flow
color = (0,255,0)  #RGB

# Read the capture and get the first frame
ret, first_frame = cap.read()

# Convert frame to Grayscale
prev_gray = cv2.cvtColor(first_frame,
                        cv2.COLOR_BGR2GRAY)

# Find the strongest corners in the first frame
prev = cv2.goodFeaturesToTrack(prev_gray,
                              mask = None,
                              **st_params)

# Create an image with the same dimensions as the frame for later drawing purposes
mask = np.zeros_like(first_frame)

# While loop
while(cap.isOpened):

    # Read the capture and get the first frame
    ret, frame = cap.read()
    
    # Convert all frame to Grayscale (previously we did only the first frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate optical flow by Lucas-Kanade
    next, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev, None, **lk_params)
    
    # Select good feature for the previous position
    good_old = prev[status==1]
    
    # Select good feature for the next position
    good_new = next[status==1]
    
    # Draw optical flow track
    for i, (new, old) in enumerate(zip(good_new, good_old)):
    
        # Return coordinates for the new point
        a,b = new.ravel()
        
        # Return coordinates for the old point
        c,d = old.ravel()
        
        # Draw line between new and old position
        mask = cv2.line(mask, 
                       (a,b),
                       (c,d),
                       color,
                       2)
        
        # Draw filled circle
        frame = cv2.circle(frame,
                          (a,b),
                          3,
                          color,
                          -1)
        
    # Overlay optical flow on original frame
    output = cv2.add(frame, mask)
    
    # Update previous frame
    prev_gray = gray.copy()
    
    # Update previous good features
    prev = good_new.reshape(-1,1,2)

    # Open new window and display the output
    cv2.imshow("Optical Flow", output)
    
    # Close the frame
    if cv2.waitKey(300) & 0xFF == 27:
        break
        
# Release and Destroy
cap.release()
cv2.destroyAllWindows()


## Vehicle Counting

In [5]:
import cv2
import numpy as np
import Car
import time

# Input and output counters
cnt_up   = 0
cnt_down = 0

cap = cv2.VideoCapture(dir1)

for i in range(19):
    print (i, cap.get(i))
cv2.ocl.setUseOpenCL(False)

h = cap.get(4)
w = cap.get(3)
frameArea = h*w
areaTH = 100
print ('Area Threshold', areaTH)

#In / out lines
line_up = int(6*(h/20))
line_down   = int(10*(h/20))

up_limit =   int(3*(h/20))
down_limit = int(14*(h/20))

print ("Red line y:",str(line_down))
print ("Blue line y:", str(line_up))
line_down_color = (255,0,0)
line_up_color = (0,0,255)
pt1 =  [0, line_down];
pt2 =  [w, int(12*(h/20))];
pts_L1 = np.array([pt1,pt2], np.int32)
pts_L1 = pts_L1.reshape((-1,1,2))
pt3 =  [0, line_up];
pt4 =  [w, int(7*(h/20))];
pts_L2 = np.array([pt3,pt4], np.int32)
pts_L2 = pts_L2.reshape((-1,1,2))

pt5 =  [0, up_limit];
pt6 =  [w, int(4*(h/20))];
pts_L3 = np.array([pt5,pt6], np.int32)
pts_L3 = pts_L3.reshape((-1,1,2))
pt7 =  [0, down_limit];
pt8 =  [w, int(16*(h/20))];
pts_L4 = np.array([pt7,pt8], np.int32)
pts_L4 = pts_L4.reshape((-1,1,2))

#Variables
font = cv2.FONT_HERSHEY_SIMPLEX
cars = []
max_p_age = 5
pid = 1
bboxes = []

subtractor = cv2.createBackgroundSubtractorMOG2(history=136 , varThreshold=50 , detectShadows=False)

while (cap.isOpened):
    ret, frame = cap.read()
    
    for i in cars:
        i.age_one() #age every object one frame
    try:
        kernel = np.ones((7, 7), np.uint8) 
        mask = subtractor.apply(frame)
        closing = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    except:
        print('EOF')
        print ('UP:',cnt_up)
        print ('DOWN:',cnt_down)
        break

    hierarchy, contours,_ = cv2.findContours(closing.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#     print(contours)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > areaTH:
            #################
            #   TRACKING    #
            #################
            
            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            x,y,w,h = cv2.boundingRect(cnt)

            new = True
            if cy in range(up_limit,down_limit):
                for i in cars:
                    if abs(cx-i.getX()) <= w and abs(cy-i.getY()) <= h:
                        # the object is close to one that was already detected before
                        new = False
                        i.updateCoords(cx,cy)   #Update coordinates on the object and resets age
                        if i.going_UP(line_down,line_up) == True:
                            cnt_up += 1;
                            print ("ID:",i.getId(),'crossed going up at',time.strftime("%c"),end='\n')
                        elif i.going_DOWN(line_down,line_up) == True:
                            cnt_down += 1;
                            print ("ID:",i.getId(),'crossed going down at',time.strftime("%c"),end='\n')
                        break
                    if i.getState() == '1':
                        if i.getDir() == 'down' and i.getY() > down_limit:
                            i.setDone()
                        elif i.getDir() == 'up' and i.getY() < up_limit:
                            i.setDone()
                    if i.timedOut():
                        index = cars.index(i)
                        cars.pop(index)
                        del i     
                if new == True:
                    p = Car.MyCar(pid,cx,cy, max_p_age)
                    cars.append(p)
                    pid += 1     
            cv2.circle(frame,(cx,cy), 5, (0,0,255), -1)
            img = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            bboxes.append([x,y,w,h])
    for i in cars:
        cv2.putText(frame, str(i.getId()),(i.getX(),i.getY()),font,0.3,i.getRGB(),1,cv2.LINE_AA)
        
    #################
    #   IMAGES      #
    #################
    str_up = 'UP: '+ str(cnt_up)
    str_down = 'DOWN: '+ str(cnt_down)
    frame = cv2.polylines(frame,[pts_L1],False,line_down_color,thickness=2)
    frame = cv2.polylines(frame,[pts_L2],False,line_up_color,thickness=2)
    frame = cv2.polylines(frame,[pts_L3],False,(255,255,255),thickness=1)
    frame = cv2.polylines(frame,[pts_L4],False,(255,255,255),thickness=1)
    cv2.putText(frame, str_up ,(10,40),font,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str_up ,(10,40),font,0.5,(0,0,255),1,cv2.LINE_AA)
    cv2.putText(frame, str_down ,(10,90),font,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str_down ,(10,90),font,0.5,(255,0,0),1,cv2.LINE_AA)
    
    cv2.imshow("mask", mask)
    cv2.imshow("Closing", closing)
    cv2.imshow("Frame", frame)

    key = cv2.waitKey(30)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

0 0.0
1 0.0
2 0.03333333333333333
3 800.0
4 480.0
5 30.0
6 842289229.0
7 1802.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0
16 0.0
17 0.0
18 0.0
Area Threshold 100
Red line y: 240
Blue line y: 144
ID: 6 crossed going up at Thu Jun 18 17:34:31 2020
ID: 6 crossed going up at Thu Jun 18 17:34:31 2020
ID: 6 crossed going up at Thu Jun 18 17:34:32 2020
ID: 22 crossed going up at Thu Jun 18 17:34:38 2020
ID: 22 crossed going up at Thu Jun 18 17:34:38 2020
ID: 22 crossed going up at Thu Jun 18 17:34:38 2020
ID: 21 crossed going down at Thu Jun 18 17:34:40 2020
ID: 18 crossed going down at Thu Jun 18 17:34:41 2020
ID: 24 crossed going down at Thu Jun 18 17:34:44 2020
ID: 43 crossed going down at Thu Jun 18 17:34:45 2020
ID: 21 crossed going up at Thu Jun 18 17:34:51 2020
ID: 42 crossed going down at Thu Jun 18 17:34:54 2020
ID: 49 crossed going down at Thu Jun 18 17:34:58 2020
ID: 52 crossed going down at Thu Jun 18 17:35:02 2020


## Summarization

inputName: the name of input video

outputName: the name of output video

Num: the number of skip frames

Seg: the number of video segmentation

In [28]:
def VideoSummary(inputName, outputName, Num, Seg):

    video = cv2.VideoCapture(inputName)
    #cv::CAP_PROP_FRAME_COUNT = 7
    total_size = int(video.get(7))
    print(total_size)
    fps = 15 #video.get(5)
    width = int(video.get(3))
    height = int(video.get(4))
    # fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')  # ffmepg -l xxx.mp4 vcodec msmpeg4v2  xxx.avi
    mode = video.get(9)
    Len = total_size / Seg
    Len_2 = (Len - 1) / Num + 1
    print('len of the prototype: %d' % (total_size))
    print('len of the summary: %d' % (Len_2))

    Out = cv2.VideoWriter(outputName, fourcc, fps, (width, height))

    for idx in range(int(Len_2)):

        if idx % 1000 == 0:
            print('idx:',idx)
        k, s = 0, 0.0
        FRAME = np.zeros((height, width, 3), dtype = float)
        while idx * Num + Len * k < total_size:
            video.set(1, idx * Num + Len * k)
            ret, frame = video.read()
            if type(frame) == type(FRAME):
                FRAME = FRAME + frame
                s += 1.0
            k += 1
        FRAME = FRAME / s
        Out.write(np.uint8(FRAME))

    video.release()
    Out.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    demo = 'Video1.avi'
    out = 'out2.avi'
    VideoSummary(demo, out, 4, 2)

1802
len of the prototype: 1802
len of the summary: 226
idx: 0


## Video Summarization using SIFT Detector

In [2]:
import cv2
import sys
import os
# import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import math
from collections import namedtuple
import time

fileName = 'Video1.avi'
folderName = 'sumsift'

skipFrames = 1

motionThreshold = 10

ColorMoments = namedtuple('ColorMoments', ['mean', 'stdDeviation', 'skewness'])

Shot = namedtuple('Shot', ['shotNumber', 'startingFrame', 'endingFrame', 'keyFrames', 'avgEntropyDiff', 'avgMotion'])

shotSifts = []

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def getEntropy(histogram, totalPixels):
    entropy = 0
    for pixels in histogram:
        if pixels != 0:
            prob = float (pixels / totalPixels)
            entropy -= prob * math.log(prob, 2)
            # print prob
    return entropy

def getColorMoments(histogram, totalPixels):
    sum = 0
    for pixels in histogram:
        sum += pixels
    mean = float (sum / totalPixels)
    sumOfSquares = 0
    sumOfCubes = 0
    for pixels in histogram:
        sumOfSquares += math.pow(pixels-mean, 2)
        sumOfCubes += math.pow(pixels-mean, 3)
    variance = float (sumOfSquares / totalPixels)
    stdDeviation = math.sqrt(variance)
    avgSumOfCubes = float (sumOfCubes / totalPixels)
    skewness = float (avgSumOfCubes**(1./3.))
    return ColorMoments(mean, stdDeviation, skewness)

def getHistogramDiff(currHistogram, prevHistogram):
    diff = 0
    for i in range(len(currHistogram)):
        diff += math.pow(currHistogram[i]-prevHistogram[i], 2)
    if diff==0:
        return 1
    else:
        return diff 

def getHistogramRatio(currHistogramDiff, prevHistogramDiff):
    ratio = float (currHistogramDiff / prevHistogramDiff)
    if ratio<1:
        ratio = 1/ratio
    return ratio

def getEuclideanDistance(currColorMoments, prevColorMoments):
    distance = math.pow(currColorMoments.mean - prevColorMoments.mean, 2) + math.pow(currColorMoments.stdDeviation - prevColorMoments.stdDeviation, 2) + math.pow(currColorMoments.skewness - prevColorMoments.skewness, 2)
    return distance

def getMotion(currImage, prevImage):
    motion = 0
    for i in range(len(currImage)):
        for j in range(len(currImage[i])):
            if int (currImage[i][j]) - int (prevImage[i][j]) > motionThreshold:
                motion += 1
    motion = float (motion / (i+1)*(j+1))
    return motion

def getSift(img):
    detector = cv2.xfeatures2d.SIFT_create()
    kp, des = detector.detectAndCompute(img, None)
    return des

def getMotionSift(img1, img2, ind):
    if ind==1:
        des1 = getSift(img1)
        shotSifts.append(des1)
    else:
        des1 = shotSifts[ind-1]
    des2 = getSift(img2)
    shotSifts.append(des2)
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    # Need to draw only good matches, so create a mask
    matchesMask = [[0,0] for i in range(len(matches))]
    # ratio test as per Lowe's paper
    sumd = 0
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.7*n.distance:
            sumd += m.distance
            matchesMask[i]=[1,0]
    return sumd

def sortShots(shots):
    maxKeyFrames = -1
    maxAvgEntropyDiff = -1
    maxAvgMotion = -1
    for shot in shots:
        if shot.keyFrames > maxKeyFrames :
            maxKeyFrames = shot.keyFrames
        if shot.avgEntropyDiff > maxAvgEntropyDiff :
            maxAvgEntropyDif = shot.avgEntropyDiff
        if shot.avgMotion > maxAvgMotion :
            maxAvgMotion = shot.avgMotion

    weights = []
    for shot in shots:
        weight = shot.keyFrames / float(maxKeyFrames) + shot.avgEntropyDiff / float(maxAvgEntropyDiff) + shot.avgMotion / float(maxAvgMotion)
        weights.append((shot.shotNumber, weight))

    print('Unsorted Weights -', weights)
    weights.sort(reverse=True, key=lambda x: x[1])
    print('Sorted Weights -', weights)
    order = [int(weight[0]) for weight in weights]
    print('Order -', order)
    return order

def save_keyframes(frame_indices, summary_frames):
    global skipFrames
    print("Saving frame indices")
    
    out_file=open(folderName+"/frame_indices_"+str(skipFrames)+".txt",'w')
    for idx in frame_indices:
        out_file.write(str(idx*skipFrames)+'\n')
    print("Saved indices")

    print("Saving frames")
    cmd="rm -r "+folderName+"/_keyframes_/"
    os.system(cmd)
    cmd="mkdir "+folderName+"_keyframes_"
    for i,frame in enumerate(summary_frames):
        cv2.imwrite(str(folderName)+"/_keyframes_/frame%d.jpg"%i, frame)
    print("Frames saved")

In [4]:
videoCap = cv2.VideoCapture(fileName)
#fps = videoCap.get(cv2.CAP_PROP_FPS)
fps=30
print("Frames per second: ", fps)

entropy = []
histogramDiff = []
histogramRatio = []
entropyDiff = []
euclideanDistance = []
motion = []

t0 = time.clock()
i = 0

success, image = videoCap.read()
height = len(image)
width = len(image[0])
print(f'height={height}, width={width}')

totalPixels = width * height
while success:

    grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # plt.imshow(grayImage, cmap = plt.get_cmap('gray'))
    # plt.show()
    histogram = cv2.calcHist([grayImage],[0],None,[256],[0,256])
    
    entropy.append( getEntropy(histogram, totalPixels) )
    
    colorMoments = getColorMoments(histogram, totalPixels)
    
    if i==0:
        histogramDiff.append(4000000)
        histogramRatio.append(200)
        entropyDiff.append(0)
        euclideanDistance.append(0)
        motion.append(0)
    else:
        histogramDiff.append(getHistogramDiff(histogram, prevHistogram) )
        if i==1:
            histogramRatio.append(1)
        else:
            histogramRatio.append( getHistogramRatio(histogramDiff[i], histogramDiff[i-1]) )
        entropyDiff.append( abs(entropy[i] - entropy[i-1]))
        euclideanDistance.append( getEuclideanDistance(colorMoments, prevColorMoments) )
        motion.append( getMotionSift(grayImage, prevGrayImage, i) )

    prevHistogram = histogram
    prevGrayImage = grayImage
    prevColorMoments = colorMoments

    i += 1
    success, image = videoCap.read()
    print(i)

meanEntropyDiff = sum(entropyDiff[1:]) / float(len(entropyDiff)-1)
meanHistogramRatio = sum(histogramRatio[1:]) / float(len(histogramRatio)-1)
meanEuclideanDistance = sum(euclideanDistance[1:]) / float(len(euclideanDistance)-1)

thresholdEntropyDiff = meanEntropyDiff
thresholdHistogramRatio = meanHistogramRatio
thresholdEuclideanDistance = meanEuclideanDistance

totalFrames = i

motionSum = 0
entropyDiffSum = 0

shots = []
shotNumber = 0
prevFrame = 0
keyFrames = 0
keyFramesArray = [0] * totalFrames
for i in range(totalFrames):
    if euclideanDistance[i] > thresholdEuclideanDistance:
        keyFrames += 1
        keyFramesArray[i] = 1

    entropyDiffSum += entropyDiff[i]
    motionSum += motion[i]

    if entropyDiff[i] > thresholdEntropyDiff and histogramRatio[i] > thresholdHistogramRatio:
        if i<= prevFrame+25 and shotNumber!=0:
            currShot = shots[shotNumber-1]
            numberOfFrames = currShot.endingFrame - currShot.startingFrame + 1
            newAvgEntropyDiff = ((currShot.avgEntropyDiff * numberOfFrames) + entropyDiffSum) / (i - currShot.startingFrame)
            newAvgMotion = ((currShot.avgMotion * numberOfFrames) + motionSum) / (i - currShot.startingFrame)
            shots[shotNumber-1] = Shot(currShot.shotNumber, currShot.startingFrame, i-1, currShot.keyFrames + keyFrames, newAvgEntropyDiff, newAvgMotion)
        else:
            avgEntropyDiff = entropyDiffSum / float(i - prevFrame)
            avgMotion = motionSum / float(i - prevFrame)
            shots.append(Shot(shotNumber, prevFrame, i-1, keyFrames, avgEntropyDiff, avgMotion))
            shotNumber += 1

        keyFrames = 0
        motionSum = 0
        entropyDiffSum = 0
        prevFrame = i

# Adding the last shot
if i!=prevFrame:
    avgEntropyDiff = entropyDiffSum / float(i - prevFrame)
    avgMotion = motionSum / float(i - prevFrame)
    shots.append(Shot(shotNumber, prevFrame, i-1, keyFrames, avgEntropyDiff, avgMotion))
    shotNumber += 1

shotsOrder = sortShots(shots)

trailerFrames = totalFrames/4
writeBit = [0] * len(shots)
for shotNo in shotsOrder:
    shot = shots[shotNo]
    shotFrames = shot.endingFrame - shot.startingFrame + 1
    if shotFrames < trailerFrames:
        writeBit[shotNo] = 1
        trailerFrames -= shotFrames
    if trailerFrames<=0:
        break

print(width, height)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
outputVideo = cv2.VideoWriter(folderName+'/summary_sift1.avi',fourcc,fps,(width,height))
inputVideo = cv2.VideoCapture(fileName)

for i in range(len(shots)):
    if writeBit[i]==1:
        j = shots[i].startingFrame * skipFrames
        while j<= shots[i].endingFrame * skipFrames:
            success, image = inputVideo.read()
            if success:
                print('writing')
                # plt.imshow(image)
                # plt.show()
                outputVideo.write(image)
            else:
                print("Error in reading video")
            j += 1
    else:
        j = shots[i].startingFrame * skipFrames
        while j<= shots[i].endingFrame * skipFrames:
            success, image = inputVideo.read()
            if not success:
                print("Error in reading video")
            j += 1

cv2.destroyAllWindows()
outputVideo.release()

print("Generating summary frames")
summary_frames=[]
keyFramesIndices=[]

for i in range(len(keyFramesArray)):
    if(keyFramesArray[i]==1):
        cmd="cp "+folderName+"/allFrames/image"+str(i)+".jpg "+folderName+"/keyFrames/"
        os.system(cmd)
        keyFramesIndices.append(i)
        summary_frames.append(cv2.imread(folderName+"/allFrames/image"+str(i)+".jpg",-1))
print("Generated Summary !")
save_keyframes(keyFramesIndices,summary_frames)



# print('Write Bit -', writeBit)
# print('Time taken to run =', time.clock() - t0, 'seconds') 
# print('len(Shots) -' , len(shots), '\n')
# print('Shots -' , shots, '\n')
# print('len(Entropy) -', len(entropy), '\n')
# print('Entropy -', entropy, '\n')
# print('len(HistogramDiff) -', len(histogramDiff), '\n')
# print('HistogramDiff -', histogramDiff, '\n')
# print('len(HistogramRatio) -', len(histogramRatio), '\n)'
# print('HistogramRatio -', histogramRatio, '\n')
# print('len(EntropyDiff) -', len(entropyDiff), '\n')
# print('EntropyDiff -', entropyDiff, '\n')
# print('len(EuclideanDistance) -', len(euclideanDistance), '\n')
# print('EuclideanDistance -', euclideanDistance, '\n')
# print('len(Motion) -', len(motion), '\n')
# print('Motion -', motion, '\n')
# print('len(MeanEntropyDiff) -', len(meanEntropyDiff), '\n')
# print('MeanEntropyDiff -', meanEntropyDiff, '\n')
# print('len(MeanHistogramRatio) -', len(meanHistogramRatio), '\n')
# print('MeanHistogramRatio -', meanHistogramRatio, '\n')
# print('len(MeanEuclideanDistance) -', len(meanEuclideanDistance), '\n')
# print('MeanEuclideanDistance -', meanEuclideanDistance, '\n')
# print('len(keyFramesArray) - ', len(keyFramesArray), '\n')
# print('# of keyFrames - ', sum(keyFramesArray), '\n')
# print('keyFramesArray - ', keyFramesArray, '\n')

Frames per second:  30
height=480, width=800
1


C:\Users\Iman Sharifi\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
writing
